In [1]:
import json
import geopandas as gpd
import pandas as pd

# Constants
PATH = '../data/location/grid.gpkg'

def combine_pois() -> gpd.GeoDataFrame:
    poi_mof = gpd.read_file('../data/demand/poi_mof.geojson', columns=['Name', 'Category']).to_crs(epsg=3826)
    poi_osm = gpd.read_file('../data/demand/poi_osm.geojson', columns=['Name', 'Category']).to_crs(epsg=3826)
    poi_mof['Source'] = pd.Series(['MOF' for i in range(len(poi_mof))])
    poi_osm['Source'] = pd.Series(['OSM' for i in range(len(poi_mof))])
    return gpd.GeoDataFrame(pd.concat([poi_mof, poi_osm]).reset_index(drop=True)[['Name', 'Category', 'Source', 'geometry']])

pois = combine_pois()
pois.to_file(PATH, driver='GPKG', layer='poi')
display(pois.head(10))
print(f'{len(pois)} rows x {len(pois.columns)} cols')

,Name,Category,Source,geometry
0,一江鎖印店,D5,MOF,POINT (303619.526 2771844.873)
1,得商有限公司,B1,MOF,POINT (304602.2 2773024.951)
2,滴二石文創有限公司,A1,MOF,POINT (304623.681 2773024.845)
3,碧達拉有限公司,C1,MOF,POINT (304623.681 2773024.845)
4,大馨花藝工作室,D5,MOF,POINT (304637.096 2773019.919)
5,嘉橙有限公司,D4,MOF,POINT (304867.453 2772823.414)
6,萬安國際股份有限公司,D1,MOF,POINT (304867.453 2772823.414)
7,洪裕國際股份有限公司,B1,MOF,POINT (304867.453 2772823.414)
8,旺普電通股份有限公司,C7,MOF,POINT (304867.453 2772823.414)
9,旺普網路資訊股份有限公司,C7,MOF,POINT (304867.453 2772823.414)


141421 rows x 4 cols


In [ ]:
# Imports
grid = gpd.read_file(PATH, layer='original-grid')
taipei = gpd.read_file(PATH, layer='taipei')

# Save to files
filtered_grid = grid[grid.intersects(taipei.iloc[0, 0])]
filtered_grid.to_file(PATH, driver='GPKG', layer='filtered-grid')

del filtered_grid, grid, taipei

In [2]:
# Calculate the base multiplier for each category in each grid cell
def get_multipliers(poi: gpd.GeoDataFrame, grid: gpd.GeoDataFrame, radius: float) -> dict:
    result: dict = {}
    poi_copy = poi.copy()
    poi_copy['geometry'] = poi_copy.buffer(radius)
    poi_copy['buffer_area'] = poi_copy.area
    poi_len = len(poi_copy)

    for poi_index, poi_item in poi_copy.iterrows():
        poi_buffer = poi_item.geometry
        category = poi_item['Category']
        candidates = gpd.sjoin(grid, gpd.GeoDataFrame(geometry=[poi_buffer], crs=grid.crs), how="inner", predicate="intersects")

        if candidates.empty:
            continue

        for grid_index, grid_row in candidates.iterrows():
            intersection_area = grid_row.geometry.intersection(poi_buffer).area
            if intersection_area <= 0:
                continue

            coverage = round((intersection_area / poi_item.buffer_area) * 1000) / 10
            if coverage <= 0:
                continue

            grid_key = str(grid_index)
            if grid_key not in result:
                result[grid_key] = {}

            result[grid_key][category] = result[grid_key].get(category, 0) + coverage

        if (poi_index + 1) % 10000 == 0:
            print(f'Computed: {poi_index + 1}/{poi_len}')

    print(f'Finished: {poi_len}/{poi_len}')
    return result

grids = gpd.read_file(PATH, layer='filtered-grid')
multipliers = get_multipliers(pois, grids, 50)
with open('../data/demand/multiplier.json', 'w', encoding='utf-8') as f:
    json.dump(multipliers, f, ensure_ascii=False)

Computed: 10000/141421
Computed: 20000/141421
Computed: 30000/141421
Computed: 40000/141421
Computed: 50000/141421
Computed: 60000/141421
Computed: 70000/141421
Computed: 80000/141421
Computed: 90000/141421
Computed: 100000/141421
Computed: 110000/141421
Computed: 120000/141421
Computed: 130000/141421
Computed: 140000/141421
Finished: 141421/141421


In [3]:
# Clean-up the JSON file
multiplier_df = pd.read_json('../data/demand/multiplier.json').sort_index().T.sort_index().apply(lambda x: round(x, 1))
display(multiplier_df.iloc[:10, :6])
print(f'{len(multiplier_df)} rows x {len(multiplier_df.columns)} cols')

,A1,A2,A3,A4,B1,B2
2,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,35.0,NaN,NaN,NaN,NaN
5,NaN,811.5,NaN,NaN,NaN,NaN
14,NaN,0.1,NaN,NaN,NaN,NaN
15,NaN,56.0,7.5,NaN,2.5,NaN
16,NaN,207.0,124.2,NaN,41.4,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,99.0,NaN,NaN,NaN,NaN
19,1.3,161.8,5.2,NaN,NaN,NaN
20,110.3,222.4,NaN,NaN,65.5,NaN


3153 rows x 24 cols
